In [1]:
import pandas as pd
import env

In [2]:
# function to deal with parsing one entry in our log data
def parse_log_entry(entry):
    parts = entry.split()
    output = {}
    output['ip'] = parts[0]
    output['timestamp'] = parts[3][1:].replace(':', ' ', 1)
    output['request_method'] = parts[5][1:]
    output['request_path'] = parts[6]
    output['http_version'] = parts[7][:-1]
    output['status_code'] = parts[8]
    output['size'] = int(parts[9])
    output['user_agent'] = ' '.join(parts[11:]).replace('"', '')
    return pd.Series(output)

url = f'mysql+pymysql://{env.username}:{env.password}@{env.hostname}/logs'
df = pd.read_sql('SELECT * FROM api_access', url)
# df = pd.concat([df.entry, df.entry.apply(parse_log_entry)], axis=1)
df = df.entry.apply(parse_log_entry)
df.head()

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,16/Apr/2019 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,16/Apr/2019 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,16/Apr/2019 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0


In [40]:
new = pd.DataFrame([
    ["95.31.18.119", "21/Apr/2019 10:02:41", "GET", "/api/v1/items/", "HTTP/1.1", '200', 1153005, "python-requests/2.21.0"],
    ["95.31.16.121", "17/Apr/2019 19:36:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '301', 1005, "python-requests/2.21.0"],
    ["97.105.15.120", "18/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '301', 2560, "python-requests/2.21.0"],
    ["97.105.19.58", "19/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '200', 2056327, "python-requests/2.21.0"],
], columns=df.columns)

df = df.append(new)

/var/folders/xf/3yzy_crd6fg3jwj3r580c2dm0000gn/T/ipykernel_55812/178743430.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new)


In [41]:
df.request_method.value_counts()

GET    13978
Name: request_method, dtype: int64

In [42]:
df.status_code.value_counts()

200    13960
499       16
301        2
Name: status_code, dtype: int64

In [43]:
df['size'].value_counts()

494819     705
3561       548
3659       291
1328       229
1787       219
          ... 
343913       1
360297       1
327529       1
278377       1
2056327      1
Name: size, Length: 191, dtype: int64

In [44]:
df['size'].mean()

450001.90578051226

In [45]:
v_low= df[df['size'] < 1000]

In [46]:
df.dtypes

ip                object
timestamp         object
request_method    object
request_path      object
http_version      object
status_code       object
size               int64
user_agent        object
dtype: object

In [47]:
v_low

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
8,97.105.19.58,16/Apr/2019 19:34:52,GET,/,HTTP/1.1,200,42,python-requests/2.21.0
34,97.105.19.58,16/Apr/2019 19:35:37,GET,/,HTTP/1.1,200,42,python-requests/2.21.0
35,97.105.19.58,16/Apr/2019 19:35:37,GET,/documentation,HTTP/1.1,200,348,python-requests/2.21.0
40,97.105.19.58,16/Apr/2019 19:35:38,GET,/,HTTP/1.1,200,42,python-requests/2.21.0
41,97.105.19.58,16/Apr/2019 19:35:39,GET,/documentation,HTTP/1.1,200,348,python-requests/2.21.0
...,...,...,...,...,...,...,...,...
12441,52.87.230.102,17/Apr/2019 02:14:28,GET,/favicon.ico,HTTP/1.1,200,162,Slackbot 1.0 (+https://api.slack.com/robots)
12577,72.181.113.170,17/Apr/2019 02:18:07,GET,/,HTTP/1.1,200,42,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...
12578,72.181.113.170,17/Apr/2019 02:18:07,GET,/favicon.ico,HTTP/1.1,200,162,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...
12598,72.181.113.170,17/Apr/2019 02:48:36,GET,/api/v1/sales?page=8,HTTP/1.1,499,0,python-requests/2.21.0


In [48]:
v_low.status_code.value_counts()

200    280
499     16
Name: status_code, dtype: int64

In [49]:
v_low.ip.value_counts()

97.105.19.58      256
72.181.105.81      14
68.201.219.223      8
72.181.113.170      4
70.121.214.34       2
173.173.113.51      2
3.92.201.136        1
52.90.165.200       1
34.207.64.242       1
54.172.14.223       1
34.229.70.250       1
35.175.171.137      1
54.145.52.184       1
35.174.209.2        1
52.91.30.150        1
52.87.230.102       1
Name: ip, dtype: int64

In [50]:
v_low_error= v_low[v_low.status_code != '200']

In [51]:
v_low_error

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
1745,97.105.19.58,16/Apr/2019 19:55:15,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.20.1
1819,97.105.19.58,16/Apr/2019 19:56:02,GET,/api/v1/sales?page=21,HTTP/1.1,499,0,python-requests/2.20.1
4242,97.105.19.58,16/Apr/2019 20:28:46,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.21.0
4405,97.105.19.58,16/Apr/2019 20:33:24,GET,/api/v1/sales?page=29,HTTP/1.1,499,0,python-requests/2.21.0
4790,97.105.19.58,16/Apr/2019 20:38:13,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0
4862,97.105.19.58,16/Apr/2019 20:38:55,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0
4871,97.105.19.58,16/Apr/2019 20:39:00,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0
5520,97.105.19.58,16/Apr/2019 20:45:17,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0
5539,97.105.19.58,16/Apr/2019 20:45:28,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.21.0
6860,97.105.19.58,16/Apr/2019 21:00:35,GET,/api/v1/sales?page=115,HTTP/1.1,499,0,python-requests/2.20.1


In [52]:
errors= df[df.status_code != '200']
errors

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
1745,97.105.19.58,16/Apr/2019 19:55:15,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.20.1
1819,97.105.19.58,16/Apr/2019 19:56:02,GET,/api/v1/sales?page=21,HTTP/1.1,499,0,python-requests/2.20.1
4242,97.105.19.58,16/Apr/2019 20:28:46,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.21.0
4405,97.105.19.58,16/Apr/2019 20:33:24,GET,/api/v1/sales?page=29,HTTP/1.1,499,0,python-requests/2.21.0
4790,97.105.19.58,16/Apr/2019 20:38:13,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0
4862,97.105.19.58,16/Apr/2019 20:38:55,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0
4871,97.105.19.58,16/Apr/2019 20:39:00,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0
5520,97.105.19.58,16/Apr/2019 20:45:17,GET,/api/v1/items,HTTP/1.1,499,0,python-requests/2.21.0
5539,97.105.19.58,16/Apr/2019 20:45:28,GET,/api/v1/sales?page=2,HTTP/1.1,499,0,python-requests/2.21.0
6860,97.105.19.58,16/Apr/2019 21:00:35,GET,/api/v1/sales?page=115,HTTP/1.1,499,0,python-requests/2.20.1


In [53]:
errors.user_agent.value_counts()

python-requests/2.21.0    14
python-requests/2.20.1     4
Name: user_agent, dtype: int64

In [54]:
 df[df.status_code == '301']

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
1,95.31.16.121,17/Apr/2019 19:36:41,GET,/api/v1/sales?page=79/,HTTP/1.1,301,1005,python-requests/2.21.0
2,97.105.15.120,18/Apr/2019 19:42:41,GET,/api/v1/sales?page=79/,HTTP/1.1,301,2560,python-requests/2.21.0


In [55]:
errors['size'].value_counts()

0       16
1005     1
2560     1
Name: size, dtype: int64

In [56]:
df.status_code.value_counts()

200    13960
499       16
301        2
Name: status_code, dtype: int64

In [57]:
errors.request_path.value_counts()

/api/v1/items             4
/api/v1/sales?page=2      3
/api/v1/sales?page=79/    2
/api/v1/sales?page=21     1
/api/v1/sales?page=29     1
/api/v1/sales?page=115    1
/api/v1/sales?page=47     1
/api/v1/sales?page=25     1
/api/v1/sales?page=80     1
/api/v1/sales?page=69     1
/api/v1/sales?page=8      1
/api/v1/sales?page=133    1
Name: request_path, dtype: int64

In [58]:
df.user_agent

0        python-requests/2.21.0
1        python-requests/2.21.0
2        python-requests/2.21.0
3        python-requests/2.21.0
4        python-requests/2.21.0
                  ...          
13973    python-requests/2.21.0
0        python-requests/2.21.0
1        python-requests/2.21.0
2        python-requests/2.21.0
3        python-requests/2.21.0
Name: user_agent, Length: 13978, dtype: object